## Creating confidence intervals in python
In this assessment, you will look at data from a study on toddler sleep habits. 

The confidence intervals you create and the questions you answer in this Jupyter notebook will be used to answer questions in the following graded assignment.

In [8]:
import numpy as np
import pandas as pd
from scipy.stats import t
import statsmodels.api as sm
pd.set_option('display.max_columns', 30) # set so can see all columns of the DataFrame

Your goal is to analyse data which is the result of a study that examined
differences in a number of sleep variables between napping and non-napping toddlers. Some of these
sleep variables included: Bedtime (lights-off time in decimalized time), Night Sleep Onset Time (in
decimalized time), Wake Time (sleep end time in decimalized time), Night Sleep Duration (interval
between sleep onset and sleep end in minutes), and Total 24-Hour Sleep Duration (in minutes). Note:
[Decimalized time](https://en.wikipedia.org/wiki/Decimal_time) is the representation of the time of day using units which are decimally related.   


The 20 study participants were healthy, normally developing toddlers with no sleep or behavioral
problems. These children were categorized as napping or non-napping based upon parental report of
children’s habitual sleep patterns. Researchers then verified napping status with data from actigraphy (a
non-invasive method of monitoring human rest/activity cycles by wearing of a sensor on the wrist) and
sleep diaries during the 5 days before the study assessments were made.


You are specifically interested in the results for the Bedtime, Night Sleep Duration, and Total 24-
Hour Sleep Duration. 

Reference: Akacem LD, Simpkin CT, Carskadon MA, Wright KP Jr, Jenni OG, Achermann P, et al. (2015) The Timing of the Circadian Clock and Sleep Differ between Napping and Non-Napping Toddlers. PLoS ONE 10(4): e0125181. https://doi.org/10.1371/journal.pone.0125181

In [2]:
# Import the data
df = pd.read_csv("nap_no_nap.csv") 

In [3]:
# First, look at the DataFrame to get a sense of the data
df.head(10)

,id,sex,age (months),dlmo time,days napped,napping,nap lights outl time,nap sleep onset,nap midsleep,nap sleep offset,nap wake time,nap duration,nap time in bed,night bedtime,night sleep onset,sleep onset latency,night midsleep time,night wake time,night sleep duration,night time in bed,24 h sleep duration,bedtime phase difference,sleep onset phase difference,midsleep phase difference,wake time phase difference
0,1,female,33.7,19.24,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.45,20.68,0.23,1.92,7.17,629.4,643.00,629.40,-1.21,-1.44,6.68,11.93
1,2,female,31.5,18.27,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.23,19.48,0.25,1.09,6.69,672.4,700.40,672.40,-0.96,-1.21,6.82,12.42
2,3,male,31.9,19.14,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.60,20.05,0.45,1.29,6.53,628.8,682.60,628.80,-0.46,-0.91,6.15,11.39
3,4,female,31.6,19.69,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.46,19.50,0.05,1.89,8.28,766.6,784.00,766.60,0.23,0.19,6.20,12.59
4,5,female,33.0,19.52,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.21,19.65,0.45,1.30,6.95,678.0,718.00,678.00,0.31,-0.13,5.78,11.43
5,6,female,36.2,18.22,4,1,14.00,14.22,15.00,15.78,16.28,93.75,137.00,19.95,20.25,0.29,1.26,6.28,602.2,653.80,695.95,-1.73,-2.03,7.05,12.06
6,7,male,36.3,19.28,1,1,14.75,15.03,15.92,16.80,16.08,106.00,80.00,20.60,20.96,0.36,2.12,7.27,618.4,655.40,724.40,-1.32,-1.68,6.84,11.99
7,8,male,30.0,21.06,5,1,13.09,13.43,14.44,15.46,15.82,121.60,163.80,22.01,22.53,0.51,2.92,7.31,526.8,582.40,648.40,-0.95,-1.47,5.86,10.25
8,9,male,33.2,19.38,2,1,14.41,14.42,15.71,17.01,16.60,155.50,131.25,20.24,20.37,0.13,1.60,6.82,626.8,660.33,782.30,-0.86,-0.99,6.22,11.44
9,10,female,37.1,19.93,3,1,13.12,13.42,14.31,15.19,15.30,106.67,130.67,20.78,21.63,0.84,2.20,6.52,549.5,626.00,656.17,-0.76,-1.82,6.21,10.59


In [4]:
df.shape

(20, 25)

In [17]:
df.columns

Index(['id', 'sex', 'age (months)', 'dlmo time', 'days napped', 'napping',
       'nap lights outl time', 'nap sleep onset', 'nap midsleep',
       'nap sleep offset', 'nap wake time', 'nap duration', 'nap time in bed',
       'night bedtime', 'night sleep onset', 'sleep onset latency',
       'night midsleep time', 'night wake time', 'night sleep duration',
       'night time in bed', '24 h sleep duration', 'bedtime phase difference',
       'sleep onset phase difference', 'midsleep phase difference',
       'wake time phase difference'],
      dtype='object')

## Hypothesis testing
We will look at two hypothesis test, each with $\alpha = .025$:  


1. Is the average bedtime for toddlers who nap later than the average bedtime for toddlers who don't nap?


$$H_0: \mu_{nap}=\mu_{no\ nap}, \ H_a:\mu_{nap}>\mu_{no\ nap}$$
Or equivalently:
$$H_0: \mu_{nap}-\mu_{no\ nap}=0, \ H_a:\mu_{nap}-\mu_{no\ nap}>0$$


2. The average 24 h sleep duration (in minutes) for napping toddlers is different from toddlers who don't nap.


$$H_0: \mu_{nap}=\mu_{no\ nap}, \ H_a:\mu_{nap}\neq\mu_{no\ nap}$$
Or equivalenty:
$$H_0: \mu_{nap}-\mu_{no\ nap}=0, \ H_a:\mu_{nap}-\mu_{no\ nap} \neq 0$$

Aside: This $\alpha$ level is equivalent to $\alpha = .05$ and then applying the [Bonferonni correction](https://en.wikipedia.org/wiki/Bonferroni_correction).

Before any analysis, we will convert 'night bedtime' into decimalized time. 

Now, isolate the column 'night bedtime' for those who nap into a new variable, and those who didn't nap into another new variable. 

In [6]:
# Convert 'night bedtime' into decimalized time
df.loc[:,'night bedtime'] = np.floor(df['night bedtime'])*60 + np.round(df['night bedtime']%1,2 )*100

## Hypothesis test one

### This is a two sided test

In [12]:
#run z test by using statsmodel 

nap_bedtime = df[df["napping"]==1]

no_nap_bedtime = df[df["napping"]==0]


nap_bedtime_new=nap_bedtime["night bedtime"].dropna()  #series one 
model_one=sm.stats.DescrStatsW(nap_bedtime_new)


no_nap_bedtime_new=no_nap_bedtime["night bedtime"].dropna() #series two 
model_two=sm.stats.DescrStatsW(no_nap_bedtime_new)

sm.stats.CompareMeans(model_one,model_two).ztest_ind(usevar='pooled') 

(2.367160052079275, 0.017925179082180703)

In [13]:
#Or
nap = df.loc[df["napping"]==1, "night bedtime"]

no_nap = df.loc[df["napping"]==0, "night bedtime"]

print(sm.stats.ttest_ind(nap, no_nap)) # prints test statistic, p-value, degrees of freedom


(2.367160052079275, 0.029334902861805394, 18.0)


### Conclusion: We could reject the null hypothesis with 95% confidence. 

## Hypothesis test two

### This is a two-sided test

In [20]:
nap = df.loc[df["napping"]==1, "night sleep duration"]

no_nap = df.loc[df["napping"]==0, "night sleep duration"]

print(sm.stats.ttest_ind(nap, no_nap)) # prints test statistic, p-value, degrees of freedom


(-2.84094280704821, 0.010840884590074436, 18.0)


### Conclusion: We could reject the null hypothesis that there is no differnce of night sleep duration between toddlers who nap and toddlers who don't 

**Question**: Given our sample size of $n$, how many degrees of freedom ($df$) are there for the associated $t$ distribution?

Now calculate the $t$-test statistic for our first hypothesis test using  
* pooled s.e.($\bar{X}_{nap} - \bar{X}_{no\ nap}$)  
* $\bar{X}_{nap} - \bar{X}_{no\ nap}$  
* $\mu_{0,\ nap} - \mu_{0,\ no\ nap}=0$, the population difference in means under the null hypothesis

**Question**: What is the p-value for the first hypothesis test?

To find the p-value, we can use the function:
```
t.cdf(y, df)
```
Which for $X \sim t(df)$ returns $P(X \leq y)$.

Because of the symmetry of the $t$ distrubution, we have that 
```
1-t.cdf(y, df)
```
returns $P(X > y)$

The function t.cdf(y, df) will give you the same value as finding the one-tailed probability of y on a t-table with the specified degrees of freedom.

Use the function t.cdf(y, df) to find the p-value for the first hypothesis test.

**Question**: What are the t-statistic and p-value for the second hypothesis test?

Calculate the $t$ test statistics and corresponding p-value using the scipy function scipy.stats.ttest_ind(a, b, equal_var=True) and check with your answer. 

**Question**: Does scipy.stats.ttest_ind return values for a one-sided or two-sided test?

**Question**: Can you think of a way to recover the results you got using 1-t.cdf from the p-value given by scipy.stats.ttest_ind?

Use the scipy function scipy.stats.ttest_ind(a, b, equal_var=True) to find the $t$ test statistic and corresponding p-value for the second hypothesis test.

In [ ]:
from scipy import stats


**Question**: For the $\alpha=.025$, do you reject or fail to reject the first hypothesis?

**Question**: For the $\alpha=.025$, do you reject or fail to reject the second hypothesis?